In [1]:
import numpy as np
import gym
import random
import keyboard
import time
import cv2

In [2]:
env = gym.make('SkiingDeterministic-v4')
action_size = env.action_space.n
print(" ".join( [f"{i}:{a}" for i,a in enumerate(env.get_action_meanings())]))

0:NOOP 1:RIGHT 2:LEFT


In [3]:
def preprocessFrame(I):
    I = I[:,:,1]
    I = I[31:103,4:76]
    I[I == 236] = 0
    I[I == 192] = 0
    I[I == 214] = 0
    I[I != 0] = 255
    return I/255

In [33]:
def get_frame_reward(I, prev):
    I = I[:,:,1]
    I = I[74:75,8:152] # Jugador 92, bandera roja 50, bandera azul 72
    if 72 not in I and 50 not in I:
        return 0
    if 72 in I:
        flags = np.where(I == 72)
    else:
        flags = np.where(I == 50)
    
    player = np.where(I == 92)[1]
    
    if len(player) == 0:
        return 1 
    
    player = player.mean()
    
    if len(flags[1]) == 2:
        if player >= flags[1][0] and player <= flags[1][1]:
            return 1
        else:
            return -1
    else:
        return prev

In [43]:
env.reset()
prev = np.zeros((72,72))
total = 0
count = 0
reward = 0

rewards = 0

prev = 0

lfc = 0
while True:
    count += 1
    action = 0
    if keyboard.is_pressed('a'):
        action = 2
    if keyboard.is_pressed('d'):
        action = 1
        
    if keyboard.is_pressed('space'):
        break
    
    next_state, reward, done, _ = env.step(action)
    total += reward

    time.sleep(0.05)
    img = next_state
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    res = get_frame_reward(next_state, prev)
    lfc += 1
    if res == 0 and prev != 0:
        print(prev, prev - 0.5*np.tanh(0.05*(lfc-25)))
        lfc = 0
        rewards += 1
    prev = res
    #print(res, f"temp/img_{str(count).rjust(4, '0')}.png")
    cv2.imwrite(f"temp/img_{str(count).rjust(4, '0')}.png", img)
    env.render()
    
    if done:
        break

print("Rewards:", rewards)
timer = 4507/count-1

flagr = 5*(20 - (-reward) // 500) if timer > 0 else 0

env.close()
print(total)
print(flagr)
print(timer)
reward = flagr + timer
print(f"FINAL: {reward}")

-1 -1.499999247602267
1 0.7497398944048823
1 1.049833997312478
1 0.901312339887548
1 0.8775406687981454
1 0.950166002687522
1 0.950166002687522
1 1.02497918747894
1 0.950166002687522
1 1.0
1 0.925557483188341
1 1.02497918747894
1 0.97502081252106
1 1.0
1 1.02497918747894
1 1.0
1 1.02497918747894
1 0.97502081252106
1 0.97502081252106
1 0.97502081252106
Rewards: 20
-4950.0
95.0
5.736920777279522
FINAL: 100.73692077727952


In [7]:
env.close()